# 3rd Iteration of Magnum Result Concatenator

Notebook containing the most up-to-date (as of 2020-03-05) methods for concatenating together a set of output datasets from the second magnum dataset. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'analysed_4'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
path_to_analysed_datasets = 'analysed_4_downsampled'
os.chdir(path_to_datasets)

---
## Saving / loading section
The metadata doesn't need to be reconstructed _every_ time, so we can store it in a .csv file output by pandas. The following 2 methods save and load, use whichever is appropriate

In [3]:
def get_dataset_metadata(path_to_analysed_datasets):
    metadata_filename = f'{path_to_analysed_datasets}_xr_metadata.csv'

    # Loading the metadata .csv file into a pandas dataframe
    try:
        analysed_infos_df = pd.read_csv(metadata_filename).set_index('adc_index')
    except:
        # If the .csv file doesn't exist, create it and save it to where we would expect it to be
        analysed_infos_df = mgut.create_analysed_ds_metadata(path_to_analysed_datasets)
        analysed_infos_df.to_csv(metadata_filename)
        
    return analysed_infos_df

In [4]:
# Joe's requested shots
# indices = [396, 398, 409, 410]

# Angle scan (H) 0.8T 
# indices = [41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63]

# Axial Scan
indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

# 100A, 5slm (He), 0.8T
# indices = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]

# 100A, 5slm (He), 1.2T
# indices = [245, 246, 247, 248, 249, 250, 251, 252, 254, 255, 256, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271]

# 110A, 5slm (H), 1.2T
# indices = [320, 321, 322, 323, 324, 325, 327, 328, 329, 330, 331, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346]

In [5]:
get_dataset_metadata('analysed_4_downsampled').loc[[116,117]]

,shot_number,shot_timestamp,shot_time,filename,time_len,sweep_len
adc_index,,,,,,
116,111,6696467906551624704,2019-05-29 16:15:32,analysed_4_downsampled/a111_116_66964679065516...,250,197
117,112,6696468713881848832,2019-05-29 16:18:40,analysed_4_downsampled/a112_117_66964687138818...,250,193


## Load adc file metadata

In [6]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
print(meta_data_ds)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_

# Creation of Dataset from adc_file index selection

In [7]:
magnum_probes = lp.MagnumProbes()

def preprocess_sel(ds, sweep=slice(0,997)):
    print(ds.time)
    ds = ds.reset_index('time', drop=True).load()
    return ds.sel(sweep=sweep)

def preprocess_autosel(ds):
    ds = ds.reset_index('time', drop=True).load()
    sweep = slice(*find_sweep_limit(ds))
    return ds.sel(sweep=sweep)

def preprocess_average(ds):
    ds = ds.reset_index('time', drop=True).load()
    sweep_min, sweep_max = find_sweep_limit(ds)

    ds_avg = ds.sel(sweep=slice(sweep_min, sweep_max)).mean('sweep')
#     ds_avg = ds_avg.assign({'d_current': 3 * ds.std('sweep')['current']})
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current'] / (10 * np.sqrt(ds['time'].size))})
    return ds_avg

def preprocess_average_downsample(ds, downsample_to=500):
    ds = ds.reset_index('time', drop=True).load()
    dsf = int(len(ds.time) / downsample_to)
    ds = ds.sel(time=slice(0, len(ds.time), dsf))
    ds_avg = ds.mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg



def find_sweep_limit(ds, probe=0):
    max_current = ds['current'].mean('direction').max('time').isel(probe=1)
    
    max_current_trim = max_current.where(exclude_outliers_cond(max_current, 1.5), drop=True)
    max_current_trim_2 = max_current_trim.where(exclude_outliers_cond(max_current_trim, 2.5), drop=True)
    
    return max_current_trim_2["sweep"].min().values, max_current_trim_2["sweep"].max().values

def exclude_outliers_cond(da, n=2):
    return np.abs(da - da.median()) < n * da.std()

## Function to get the dataset of interest from a set of indices

This is constructed from the boxes in result_concatenation_2.ipynb 

In [8]:
def get_dataset_from_indices(indices, parallel=True, preprocess='average', load_fl=True, anglescan_fl=True, average_direction_fl=True, path_to_analysed_datasets='analysed_2'):
    # Load a selection of useful arrays / DataArrays
    analysed_metadata_df = get_dataset_metadata(path_to_analysed_datasets)
    analysed_metadata_oi = analysed_metadata_df.loc[indices]

    files_oi = analysed_metadata_oi['filename'].values
    shot_numbers = analysed_metadata_oi['shot_number'].values
    shot_numbers_da = xr.DataArray(shot_numbers, dims=['shot_number'], name='shot_number')
    min_sweep_len = analysed_metadata_oi['sweep_len'].min()

    # Select meta data for shots of interest
    meta_data_oi_ds = meta_data_ds.sel(shot_number=shot_numbers)
    
    # Construct an additional DataArray for the probe tilt - useful for getting anglescan info
    tilt_da = meta_data_oi_ds[['shot_tilt', 'adc_4_probe', 'adc_5_probe']]
    tilt_da['shot_tilt'] = tilt_da.shot_tilt.round()
    
    if preprocess == 'average':
        combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_average, parallel=parallel, combine='nested')
    elif preprocess == 'average_downsample':
        combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_average_downsample, parallel=parallel, combine='nested')
    elif preprocess == 'autosel':
        combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_autosel, parallel=parallel, combine='nested')
    elif preprocess == 'sel':
        combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=lambda x: preprocess_sel(x, slice(0,min_sweep_len)), parallel=parallel, combine='nested')
    else:
        raise ValueError('Invalid preprocess function specified.')
    
    if load_fl:
        combined_ds = combined_ds.load()

    # Then merge it with the metadata for a complete dataset of the desired shots
    combined_ds = xr.merge([combined_ds, meta_data_oi_ds.rename({'shot_time':'sweep_time'})]).assign_coords(tilt=tilt_da['shot_tilt'])
    
    if anglescan_fl:
        anglescan_ds = combined_ds.swap_dims({'shot_number':'tilt'}) #.mean('direction')
                
        # Reorganise to make tilt and probe dimensions
        probes_1 = anglescan_ds.sel(probe=['S', 'L']).groupby('tilt').mean() #.where(np.isfinite(anglescan_ds.voltage), drop=True) 
        probes_2 = anglescan_ds.sel(probe=['R', 'B']).groupby('tilt').mean() #.where(np.isfinite(anglescan_ds.voltage), drop=True)

        combined_ds = xr.concat([probes_1, probes_2], dim='probe')
    
    if average_direction_fl:
        combined_ds = combined_ds.mean('direction')
    
    return combined_ds, tilt_da

In [9]:
def plot_anglescan_IVs(anglescan_ds, sup_title=None):    
    iv_fig, iv_ax = plt.subplots(2, 2, sharex=True)
    for i, probe in enumerate(anglescan_ds['probe'].values):
        iv_ax[i%2][i//2].set_title(probe)

        ds = anglescan_ds.sel(probe=probe)
        ds.set_coords('voltage')['current'].plot.line(ax=iv_ax[i%2][i//2], x='voltage', hue='tilt')
        iv_ax[i%2][i//2].axhline(0, linestyle='--', color='grey', linewidth=1)
        iv_ax[i%2][i//2].set_ylabel('I (A)')
        iv_ax[i%2][i//2].set_xlabel(r'$V_P$ (V)')

    iv_fig.suptitle(f'IV characteristics for {sup_title}')
    plt.show()
    
def plot_anglescan_IVs_vertical(anglescan_ds):
    fig, ax = plt.subplots(4, sharex=True, sharey=True)
    anglescan_ds.set_coords('voltage').sel(probe='L')['current'].plot.line(x='voltage', hue='tilt', ax=ax[0])
    anglescan_ds.set_coords('voltage').sel(probe='S')['current'].plot.line(x='voltage', hue='tilt', ax=ax[1])
    anglescan_ds.set_coords('voltage').sel(probe='B')['current'].plot.line(x='voltage', hue='tilt', ax=ax[2])
    anglescan_ds.set_coords('voltage').sel(probe='R')['current'].plot.line(x='voltage', hue='tilt', ax=ax[3])
    ax[0].axhline(0, linestyle='--', color='grey', linewidth=1)
    ax[1].axhline(0, linestyle='--', color='grey', linewidth=1)
    ax[2].axhline(0, linestyle='--', color='grey', linewidth=1)
    ax[3].axhline(0, linestyle='--', color='grey', linewidth=1)
    plt.show()
    
def plot_anglescan_IV_trace(anglescan_ds, probe='S', sup_title='S probe'):
    fig, ax = plt.subplots(2, sharex=True)
    anglescan_ds.sel(probe='S')['voltage'].plot.line(x='time', hue='tilt', ax=ax[0])
    anglescan_ds.sel(probe='S')['current'].plot.line(x='time', hue='tilt', ax=ax[1])
    
    fig.suptitle(f'IV traces for {sup_title}')
    plt.show()

In [10]:
def plot_anglescan_averaged_ts(anglescan_ds, sup_title=None):
    ts_temp = anglescan_ds.ts_temperature.mean(['probe', 'tilt'])
    ts_d_temp = anglescan_ds.ts_temperature.std(['probe', 'tilt'])
    ts_dens = anglescan_ds.ts_density.mean(['probe', 'tilt'])
    ts_d_dens = anglescan_ds.ts_density.std(['probe', 'tilt'])

    fig, ax = plt.subplots(2)
    ax[0].set_title('Temperature')
    ax[1].set_title('Density')
    # ax[0].errorbar(helium_anglescan_ds.ts_radial_pos, ts_temp, yerr=helium_anglescan_ds.ts_d_temperature.mean('shot_number'), color='silver', ecolor='silver')
    ax[0].errorbar(anglescan_ds.ts_radial_pos, ts_temp, yerr=ts_d_temp)
    ax[1].errorbar(anglescan_ds.ts_radial_pos, ts_dens, yerr=ts_d_dens)

    for i in [0, 1]:
        for probe_pos in [-6, 4, 14, 24]:
            ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
    plt.show()
    fig.suptitle(f'Thomson Scattering profiles for {sup_title}')
    
    
def plot_anglescan_multi_ts(anglescan_ds, ax=None, sup_title=None):
    ts_temp = anglescan_ds['ts_temperature'].sel(probe='S')
    ts_d_temp = anglescan_ds['ts_d_temperature'].sel(probe='S')
    ts_dens = anglescan_ds['ts_density'].sel(probe='S')
    ts_d_dens = anglescan_ds['ts_d_density'].sel(probe='S')
    
    if ax is None:
        fig, ax = plt.subplots(2)
    
    ax[0].set_title('Temperature')
    ax[1].set_title('Density')
    ts_temp.plot.line(hue='tilt', ax=ax[0])
    ts_dens.plot.line(hue='tilt', ax=ax[1])

    for i in [0, 1]:
        for probe_pos in [-6, 4, 14, 24]:
            ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
    plt.show()
    
    plt.suptitle(f'Thomson Scattering profiles for {sup_title}')

In [11]:
# copied from concatted_ds_analysis_0.ipynb and joes_analysis.ipynb
import collections

def fit_anglescan_ds(anglescan_ds, probes=('L', 'S', 'B', 'R'), ax=True, scan_param='tilt', threshold=1):
    mandatory_labels = [
        scan_param,
        'probe', 
        'B',
        'ts_temp',
        'ts_dens',
        'fit_success_fl',
    ]
    fit_param_labels = [
        'temp',
        'd_temp',
        'isat', 
        'd_isat',
        'a',
        'd_a',
        'v_f',
        'd_v_f', 
        'dens', 
        'd_dens',
        'chi2',
        'reduced_chi2'
    ]
    all_labels = mandatory_labels + fit_param_labels
    fit_df = pd.DataFrame(columns=all_labels)

    for tilt in anglescan_ds[scan_param].values:
        tilt_ds = anglescan_ds.sel(**{scan_param: tilt})
        for probe in probes:
            probe_tilt_ds = tilt_ds.sel(probe=probe)
            probe_tilt_ds = probe_tilt_ds.where(np.isfinite(probe_tilt_ds['voltage']), drop=True)
            probe_tilt_ds = probe_tilt_ds.where(np.isfinite(probe_tilt_ds['current']), drop=True)
            
            iv_indices = np.where(probe_tilt_ds.current < 0)[0]
            if len(iv_indices) >= 1 and iv_indices[0] == 0:
                extreme_index = max(iv_indices)
                extension = [extreme_index + (i + 1) for i in range(threshold) if extreme_index + (i + 1) < len(probe_tilt_ds.current)]
                ext_iv_indices = np.concatenate((iv_indices, extension)).astype(np.int64)
            elif len(iv_indices) >= 1:
                extreme_index = min(iv_indices)
                extension = [extreme_index - (threshold - i) for i in range(threshold) if extreme_index - (threshold - i) >= 0]
                ext_iv_indices = np.concatenate((extension, iv_indices)).astype(np.int64)
            else:
                ext_iv_indices = iv_indices
            probe_tilt_ds = probe_tilt_ds.isel(time=ext_iv_indices)
            
            if scan_param == 'tilt':
                alpha = np.radians(tilt)
            else:
                alpha = np.radians(probe_tilt_ds['tilt'].values)

            if len(probe_tilt_ds.time) == 0:
                print('Time array has no length, continuing...')
                continue

            shot_iv = iv.IVData(probe_tilt_ds['voltage'].values,
                                -probe_tilt_ds['current'].values,
                                probe_tilt_ds['shot_time'].values,
                                sigma=probe_tilt_ds['d_current'].values)

            

            fit_params = {}
            fitter = fts.FullIVFitter()
            try:
    #             shot_fit = shot_iv.multi_fit(sat_region=-40)
                shot_fit = fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
        
                dens = magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=alpha)
                d_dens = magnum_probes[probe].get_d_density(
                    shot_fit.get_isat(), 
                    shot_fit.get_isat_err(), 
                    shot_fit.get_temp(), 
                    shot_fit.get_temp_err(),
                    alpha=alpha
                )
                if isinstance(dens, collections.Iterable):
                    dens = dens[0]
                    d_dens = d_dens[0]
        
                fit_params = {
                    'fit_success_fl': True,
                    'temp': shot_fit.get_temp(),
                    'd_temp': shot_fit.get_temp_err(),
                    'isat': shot_fit.get_isat(), 
                    'd_isat': shot_fit.get_isat_err(),
                    'a': shot_fit.get_sheath_exp(), 
                    'd_a': shot_fit.get_sheath_exp_err(),
                    'v_f': shot_fit.get_floating_pot(),
                    'd_v_f': shot_fit.get_floating_pot_err(),
                    'dens': dens,
                    'd_dens': d_dens,
                    'chi2': shot_fit.chi2,
                    'reduced_chi2': shot_fit.reduced_chi2,
                }
                if ax is not None:
                    if ax is True:
                        fig, ax = plt.subplots()
                    ax.errorbar(shot_iv['V'], shot_iv['I'], yerr=shot_iv['sigma'], 
                                ecolor='silver', color='silver', marker='+', zorder=1)
                    ax.plot(*shot_fit.get_fit_plottables())
            except RuntimeError as e:
                print(f'WARNING: Failed on tilt {tilt} with probe {probe}')
                fit_params = {label: np.NaN for label in fit_param_labels}
                fit_params['fit_success_fl'] = False


            fit_df = fit_df.append({
                scan_param: tilt,
                'probe': probe, 
                'B': np.around(probe_tilt_ds['shot_b_field'].max().values, decimals=1),
                'ts_temp': probe_tilt_ds['ts_temperature'].max().values,
                'ts_dens': probe_tilt_ds['ts_density'].max().values,
                **fit_params,
            }, ignore_index=True)
    
    return fit_df

def fit_densscan_ds(densscan_ds, probes=('L', 'S'), threshold=0.15, ax=True):
    densscan_ds = densscan_ds.max('ts_radial_pos')
    mandatory_labels = [
        'shot',
        'probe', 
        'B',
        'ts_temp',
        'ts_dens',
        'shot_time',
        'fit_success_fl',
    ]
    fit_param_labels = [
        'temp',
        'd_temp',
        'isat', 
        'd_isat',
        'a',
        'd_a',
        'v_f',
        'd_v_f', 
        'dens', 
        'd_dens',
        'chi2',
        'reduced_chi2'
    ]
    all_labels = mandatory_labels + fit_param_labels
    fit_df = pd.DataFrame(columns=all_labels)

    for shot in densscan_ds.shot_number.values:
        shot_ds = densscan_ds.sel(shot_number=shot)
        for probe in probes:
            probe_shot_ds = shot_ds.sel(probe=probe)
            probe_shot_ds = probe_shot_ds.where(np.isfinite(probe_shot_ds['voltage']), drop=True)
            probe_shot_ds = probe_shot_ds.where(np.isfinite(probe_shot_ds['current']), drop=True)
            probe_shot_ds = probe_shot_ds.where(probe_shot_ds.current < threshold, drop=True)

            alpha = np.radians(probe_shot_ds['tilt'].values)

            if len(probe_shot_ds.time) == 0:
                print('Time has no length, continuing...')
                continue
    #         probe_shot_ds = probe_shot_ds

    #         print(probe_shot_ds)
            shot_iv = iv.IVData(probe_shot_ds['voltage'].values,
                                -probe_shot_ds['current'].values,
                                probe_shot_ds['shot_time'].values,
                                sigma=probe_shot_ds['d_current'].values)

            if ax is not None:
                if ax is True:
                    fig, ax = plt.subplots()
                ax.errorbar(shot_iv['V'], shot_iv['I'], yerr=shot_iv['sigma'], 
                            ecolor='silver', color='silver', marker='+', zorder=1)

            fit_params = {}
            fitter = fts.FullIVFitter()
            try:
    #             shot_fit = shot_iv.multi_fit(sat_region=-40)
                shot_fit = fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
                                         
                                
                fit_params = {
                    'fit_success_fl': True,
                    'temp': shot_fit.get_temp(),
                    'd_temp': shot_fit.get_temp_err(),
                    'isat': shot_fit.get_isat(), 
                    'd_isat': shot_fit.get_isat_err(),
                    'a': shot_fit.get_sheath_exp(), 
                    'd_a': shot_fit.get_sheath_exp_err(),
                    'v_f': shot_fit.get_floating_pot(),
                    'd_v_f': shot_fit.get_floating_pot_err(),
                    'dens': magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=alpha)[0],
                    'd_dens': magnum_probes[probe].get_d_density(
                        shot_fit.get_isat(), 
                        shot_fit.get_isat_err(), 
                        shot_fit.get_temp(), 
                        shot_fit.get_temp_err(),
                        alpha=alpha,
                    )[0],
                    'chi2': shot_fit.chi2,
                    'reduced_chi2': shot_fit.reduced_chi2,
                }
                ax.plot(*shot_fit.get_fit_plottables())
            except RuntimeError as e:
                print(f'WARNING: Failed on shot {shot} with probe {probe}')
                fit_params = {label: np.NaN for label in fit_param_labels}
                fit_params['fit_success_fl'] = False


            fit_df = fit_df.append({
                'shot': shot,
                'probe': probe, 
                'B': np.around(probe_shot_ds['shot_b_field'].mean().values, decimals=1),
                'ts_temp': probe_shot_ds['ts_temperature'].mean().values,
                'ts_dens': probe_shot_ds['ts_density'].mean().values,
                'shot_time': probe_shot_ds['ts_time'].values,
                **fit_params
            }, ignore_index=True)
    
    return fit_df

## Different Applications of this Dataset
This is after having been fed different indices, and then different plots constructed
This is, firstly, the anglescans

---

### Low field hydrogen shot (0.8T, H) 
Higher density anglescan

In [1]:
super_title = 'H shot @ 0.8T'
indices_08T_H = [41,42,43,44,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63]

In [14]:
angle_scan_ds, tilt_da = get_dataset_from_indices(indices_08T_H, anglescan_fl=True, path_to_analysed_datasets=path_to_analysed_datasets)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [14]:
# plot_anglescan_IVs_vertical(angle_scan_ds)

In [17]:
plot_anglescan_IV_trace(angle_scan_ds)

In [70]:
plot_anglescan_IVs(angle_scan_ds, sup_title='H shot @ 0.8T')

In [16]:
plot_anglescan_averaged_ts(angle_scan_ds, sup_title='H shot @ 0.8T')

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [79]:
plot_anglescan_multi_ts(angle_scan_ds, sup_title='H shot @ 0.8T')

In [87]:
# this is currently now working...
fit_df = fit_anglescan_ds(angle_scan_ds, ax=True, threshold=2)

/home/jleland/coding/projects/flopter/flopter/core/lputils.py:157: RuntimeWarning: divide by zero encountered in double_scalars
  / (np.sin(theta_p) + (np.tan(theta_perp) * np.cos(theta_p)))).clip(min=0)
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:112: RuntimeWarning: invalid value encountered in arccos
  theta_c = max(2 * np.arccos((self.radius - d) / self.radius), 0)
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:114: RuntimeWarning: invalid value encountered in double_scalars
  h_r = (self.radius - d) * np.sin(alpha)
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:112: RuntimeWarning: invalid value encountered in arccos
  theta_c = max(2 * np.arccos((self.radius - d) / self.radius), 0)
/home/jleland/coding/projects/flopter/flopter/core/fitters.py:216: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + (a * np.float_power(np.absolute(V), [0.75])))


In [22]:
# Formulation of technique for trimming down to the correct IV characteristic
for tilt in angle_scan_ds.tilt.values:
    ds = angle_scan_ds.sel(probe='L', tilt=tilt).max('ts_radial_pos')

    ds = ds.where(np.isfinite(ds['voltage']), drop=True)
    ds = ds.where(np.isfinite(ds['current']), drop=True)
    threshold = 2

    iv_indices = np.where(ds.current < 0)[0]
    if len(iv_indices) >= 1 and iv_indices[0] == 0:
        extreme_index = max(iv_indices)
        extension = [extreme_index + (i + 1) for i in range(threshold) if extreme_index + (i + 1) < len(ds.current)]
        print(extreme_index, extension)
        ext_iv_indices = np.concatenate((iv_indices, extension)).astype(np.int64)
    elif len(iv_indices) >= 1:
        extreme_index = min(iv_indices)
        extension = [extreme_index - (threshold - i) for i in range(threshold) if extreme_index - (threshold - i) >= 0]
        print(extreme_index, extension)
        ext_iv_indices = np.concatenate((extension, iv_indices)).astype(np.int64)
    else:
        ext_iv_indices = iv_indices
        
    print(np.where(ds.current < 0))
#     print(ext_iv_indices)
    orig_trim_ds = ds.isel(time=iv_indices)
    trim_ds = ds.isel(time=ext_iv_indices)
    
    fig, ax = plt.subplots()
    ds.set_coords('voltage')['current'].plot.line(x='voltage')
    trim_ds.set_coords('voltage')['current'].plot.line(x='voltage')
    orig_trim_ds.set_coords('voltage')['current'].plot.line(x='voltage')
    ax.axhline(y=0, **c.AX_LINE_DEFAULTS)

    ds

33 [34, 35]
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]),)
36 [37, 38]
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36]),)
38 [39, 40]
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38]),)
38 [39, 40]
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38]),)
10 [8, 9]
(array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49]),)
10 [8, 9]
(array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22

In [102]:
# for i, row in fit_df.iterrows():
#     if row['probe'] != 'R':
#         continue
# #     print(i, row['tilt'], row['probe'], row['isat'], row['dens'])
#     print(f"{row['tilt']}, ({row['isat']:.2g}) dens = {magnum_probes[row['probe']].get_density(row['isat'], row['temp'], alpha=np.radians(row['tilt']))}")

In [88]:
fit_df

,tilt,probe,B,ts_temp,ts_dens,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f,dens,d_dens,chi2,reduced_chi2
0,-0.0,L,0.8,1.1582224942232977,2.1969938209699162e+20,True,0.528507,0.941233,0.001572,0.010414,2.161044,11.687511,-8.197170,0.038092,6.876299e+17,4.596515e+18,201338.709713,5162.531018
1,-0.0,S,0.8,1.1582224942232977,2.1969938209699162e+20,True,2.579392,0.179316,0.017278,0.000670,0.074894,0.007750,-8.350473,0.116657,9.546003e+18,3.199718e+18,5549.139907,142.285639
2,-0.0,B,0.8,1.1582224942232977,2.1969938209699162e+20,True,1.088586,0.020765,0.043897,0.000472,0.014131,0.001052,-8.333964,0.019564,6.535807e+18,9.904113e+17,1528.045158,39.180645
3,-0.0,R,0.8,1.1582224942232977,2.1969938209699162e+20,True,1.843743,0.569360,0.000044,0.000035,5.512839,3.150924,-19.657813,0.413812,NaN,NaN,1147.078204,32.773663
4,1.0,L,0.8,1.2197398517553808,2.338641880874903e+20,True,2.251085,0.068790,0.092221,0.002329,0.036213,0.003807,-9.380995,0.034529,1.634102e+19,2.209564e+18,3950.483685,101.294453
5,1.0,S,0.8,1.2197398517553808,2.338641880874903e+20,True,3.017231,0.223454,0.070001,0.003837,0.038613,0.010740,-8.282177,0.074899,2.526910e+19,6.303345e+18,22641.766345,580.558111
6,1.0,B,0.8,1.2197398517553808,2.338641880874903e+20,True,1.174110,0.016542,0.043610,0.000292,0.013957,0.000507,-8.489245,0.016168,5.259912e+18,6.718831e+17,941.579389,24.143061
7,1.0,R,0.8,1.2197398517553808,2.338641880874903e+20,True,2.952287,0.666716,0.000083,0.000043,4.032628,1.451135,-21.931373,0.451617,NaN,NaN,1772.044802,50.629851
8,2.0,L,0.8,1.177180641835639,2.3412117474122714e+20,True,2.802673,0.033835,0.151773,0.001630,0.070955,0.001870,-9.229724,0.010147,2.068759e+19,2.369158e+18,569.150520,14.593603
9,2.0,S,0.8,1.177180641835639,2.3412117474122714e+20,True,2.862495,0.154824,0.138384,0.005201,0.025505,0.006724,-8.925142,0.056081,3.941155e+19,7.653674e+18,23190.295454,594.622960


In [28]:
fitter = fts.FullIVFitter()
colours = [['red', 'orangered'], ['blue', 'purple'], ['orange', 'gold'], ['green', 'lime']]

for index, row in fit_df.iterrows():
    i = index % 4
    if i == 0:
        fig, ax = plt.subplots()
    
    label = f'{row["probe"]}_{row["ts_dens"]:.2g}_{{}}'
    
    print(row['probe'], row['tilt'])
    ds = angle_scan_ds.sel(tilt=row['tilt'], probe=row['probe']).max('ts_radial_pos')
    ds = ds.where(ds.current < 0.01, drop=True)
    
#     (-ds.set_coords('voltage')['current']).plot.line(x='voltage', ax=ax, label=label.format('data'))
    ax.errorbar(ds['voltage'], -ds['current'].values, yerr=ds['d_current'].values, 
                label=label.format('data'), ecolor='silver', color=colours[i][0])
    
    fit_params = [
        row['isat'],
        row['a'],
        row['temp'],
        row['v_f']
    ]
    fit_reconstruction = fitter.fit_function(ds['voltage'].values, *fit_params)
    ax.plot(ds['voltage'].values, fit_reconstruction, label=label.format('fit'), color=colours[i][1])  
    ax.legend()
    ax.set_title(ds.tilt.values)
#     ax.set_ylim(-0.1)

L 0.0
S 0.0
B 0.0
R 0.0
L 1.0


/home/jleland/coding/projects/flopter/flopter/core/fitters.py:216: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + (a * np.float_power(np.absolute(V), [0.75])))


S 1.0
B 1.0
R 1.0
L 2.0
S 2.0
B 2.0
R 2.0
L 3.0
S 3.0
B 3.0
R 3.0
L 4.0
S 4.0
B 4.0
R 4.0
L 5.0
S 5.0
B 5.0
R 5.0
L 6.0
S 6.0
B 6.0
R 6.0
L 7.0
S 7.0
B 7.0
R 7.0
L 8.0
S 8.0
B 8.0
R 8.0
L 9.0
S 9.0
B 9.0
R 9.0
L 10.0
S 10.0
B 10.0
R 10.0


In [82]:
ds = fit_df.to_xarray().set_coords(['probe', 'tilt']).swap_dims({'index':'tilt'}).drop('index')


In [83]:
s_shots = ds.where(ds.probe=='S', drop=True).drop('probe')
l_shots = ds.where(ds.probe=='L', drop=True).drop('probe')
b_shots = ds.where(ds.probe=='B', drop=True).drop('probe')
r_shots = ds.where(ds.probe=='R', drop=True).drop('probe')

probes_da = xr.DataArray(['L', 'S', 'B', 'R'], dims=['probe'], name='probe')

ds_stacked = xr.concat([l_shots, s_shots, b_shots, r_shots], dim=probes_da)
ds_stacked

<xarray.Dataset>
Dimensions:         (probe: 4, tilt: 11)
Coordinates:
  * tilt            (tilt) float64 -0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * probe           (probe) <U1 'L' 'S' 'B' 'R'
Data variables:
    B               (probe, tilt) float64 0.8 0.8 0.8 0.8 ... 0.8 0.8 0.8 0.8
    ts_temp         (probe, tilt) object 1.1582224942232977 ... 1.1947671155816273
    ts_dens         (probe, tilt) object 2.1969938209699162e+20 ... 2.279034452822469e+20
    fit_success_fl  (probe, tilt) object True True True True ... True True True
    temp            (probe, tilt) float64 0.5285 2.251 2.803 ... 2.968 3.244
    d_temp          (probe, tilt) float64 0.9412 0.06879 ... 0.2044 0.177
    isat            (probe, tilt) float64 0.001572 0.09222 ... 0.003748 0.004955
    d_isat          (probe, tilt) float64 0.01041 0.002329 ... 0.0001556
    a               (probe, tilt) float64 2.161 0.03621 ... 0.08057 0.06867
    d_a             (probe, tilt) float64 11.69 0.003807 ... 0.007325 0.

In [75]:
ts_temp = angle_scan_ds.ts_temperature.mean(['probe', 'tilt'])
ts_d_temp = angle_scan_ds.ts_temperature.std(['probe', 'tilt'])
ts_dens = angle_scan_ds.ts_density.mean(['probe', 'tilt'])
ts_d_dens = angle_scan_ds.ts_density.std(['probe', 'tilt'])

probe_pos_uncertainty = 1.5
ts_probe_temps = {}
ts_probe_denss = {}
for probe, probe_pos in magnum_probes.probe_position.items():
    probe_pos_limits = [probe_pos, probe_pos - probe_pos_uncertainty, probe_pos + probe_pos_uncertainty]
    ts_probe_temps[probe.upper()] = ts_temp.interp(ts_radial_pos=probe_pos_limits).values
    ts_probe_denss[probe.upper()] = ts_dens.interp(ts_radial_pos=probe_pos_limits).values


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [76]:
fig, ax = plt.subplots(4, sharex=True)

for probe in ds_stacked.probe.values:
    if probe == 'R':
        continue
    ax[0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_stacked.sel(probe=probe), label=probe)
    ax[1].errorbar('tilt', 'isat', yerr='d_isat', data=ds_stacked.sel(probe=probe), label=probe)
    ax[2].errorbar('tilt', 'dens', yerr='d_dens', data=ds_stacked.sel(probe=probe), label=probe)
    ax[3].plot('tilt', 'reduced_chi2', data=ds_stacked.sel(probe=probe), label=probe)

ax[0].set_ylabel('T / eV')
ax[1].set_ylabel(r'$I_{sat}$ / A')
ax[2].set_ylabel(r'$n_e$ / m$^{-3}$')
ax[3].set_ylabel(r'$\chi^2_{\nu}$')

ax[0].legend()
ax[1].legend()

In [84]:
fig, ax = plt.subplots(3, sharex=True)

# The default colours from pyplot
tableau_palette = ['tab:blue', 'tab:orange', 'tab:green']

for i, probe in enumerate(ds_stacked.probe.values):
    if probe == 'R' or probe == 'B':
        continue
    ax[0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_stacked.sel(probe=probe), label=probe)
    ax[1].errorbar('tilt', 'dens', yerr='d_dens', data=ds_stacked.sel(probe=probe), label=probe)
    ax[2].plot('tilt', 'reduced_chi2', data=ds_stacked.sel(probe=probe), label=probe)
    
    # Plot the ts values at the probe positions using interpolaton
    ax[0].axhline(y=ts_probe_temps[probe][0], color=tableau_palette[i], 
                  linewidth=1, linestyle='dashed', label=f'TS-{probe}')
#     ax[0].axhspan(ts_probe_temps[probe][1], ts_probe_temps[probe][2], color=tableau_palette[i], alpha=0.25)
    ax[1].axhline(y=ts_probe_denss[probe][0], color=tableau_palette[i], 
                  linewidth=1, linestyle='dashed', label=f'TS-{probe}')
#     ax[1].axhspan(ts_probe_denss[probe][1], ts_probe_denss[probe][2], color=tableau_palette[i], alpha=0.25)

ax[0].set_ylabel('T / eV')
ax[1].set_ylabel(r'$n_e$ / m$^{-3}$')
ax[2].set_ylabel(r'$\chi^2_{\nu}$')
ax[2].set_xlabel(r'probe tilt / $^{\circ}$')


# Plot the reduced chi^2 on a log plot and indicate where 1 is
ax[2].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
# ax[2].set_ylim(0, 3)
ax[2].set_yscale('log')

# ax[0].legend()
# ax[1].legend()
ax[2].legend()

In [35]:
fig, ax = plt.subplots(2)

for probe in ds_stacked.probe.values:
    ax[0].errorbar('tilt', 'isat', yerr='d_isat', data=ds_stacked.sel(probe=probe), label=probe)
    ax[1].plot('tilt', 'reduced_chi2', data=ds_stacked.sel(probe=probe), label=probe)
    
ax[0].legend()
ax[1].legend()

In [36]:


fig, ax = plt.subplots(2, sharex=True)
ax[0].set_ylabel('T / eV')
ax[1].set_ylabel(r'$n_e$ / m$^{-3}$')
# ax[0].errorbar(helium_anglescan_ds.ts_radial_pos, ts_temp, yerr=helium_anglescan_ds.ts_d_temperature.mean('shot_number'), color='silver', ecolor='silver')
ax[0].errorbar(angle_scan_ds.ts_radial_pos, ts_temp, yerr=ts_d_temp, color='k')
ax[1].errorbar(angle_scan_ds.ts_radial_pos, ts_dens, yerr=ts_d_dens, color='k')


for probe, probe_pos in magnum_probes.probe_position.items():
#         ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
    if probe == 'r':
        continue
    ds_probe = ds_stacked.sel(probe=probe.upper())
    ax[0].errorbar(probe_pos * np.ones_like(ds_probe['tilt'].values), ds_probe['temp'].values, 
                   yerr=ds_probe['d_temp'], xerr=2.5, 
                   linestyle='none', label=probe)
    ax[1].errorbar(probe_pos * np.ones_like(ds_probe['tilt'].values), ds_probe['dens'].values, 
                   yerr=ds_probe['d_dens'], xerr=2.5, 
                   linestyle='none', label=probe)
        
plt.legend()
plt.show()
fig.suptitle(f'Thomson Scattering profiles with probe fits overlaid')

Text(0.5, 0.98, 'Thomson Scattering profiles with probe fits overlaid')

---
### Medium field hydrogen shot (1.2T, H)

In [17]:
indices_12T_H = indices = [320, 321, 322, 323, 324, 325, 327, 328, 329, 330, 331, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346]
super_title = 'H shot @ 1.2T'

In [18]:
angle_scan_ds, tilt_da = get_dataset_from_indices(indices_12T_H, anglescan_fl=True)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [19]:
# lowfield_hydrogen_ds = lowfield_hydrogen_ds.set_coords('voltage')
angle_scan_ds

<xarray.Dataset>
Dimensions:                 (probe: 4, tilt: 11, time: 50, ts_radial_pos: 46)
Coordinates:
  * tilt                    (tilt) float64 -0.0 1.0 2.0 3.0 ... 7.0 8.0 9.0 10.0
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * probe                   (probe) object 'S' 'L' 'R' 'B'
Dimensions without coordinates: time
Data variables:
    voltage                 (tilt, probe, time) float64 -94.64 -92.76 ... -2.224
    current                 (tilt, probe, time) float64 -0.0114 ... 0.6191
    shot_time               (tilt, probe, time) float64 8.473 8.473 ... 10.7
    start_time              (tilt, probe) float64 8.471 8.471 ... 10.7 10.7
    d_current               (tilt, probe, time) float64 0.0003219 ... 0.01477
    ts_density              (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (probe, tilt, ts_radial_pos) float64 nan 

In [16]:
plot_anglescan_IVs(angle_scan_ds, sup_title=super_title)

In [21]:
plot_anglescan_averaged_ts(angle_scan_ds, sup_title=super_title)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [17]:
plot_anglescan_multi_ts(angle_scan_ds, sup_title=super_title)

---
### High field hydrogen shot (1.5T, H) 
The one on all 4 probes

In [25]:
indices_15T_H = [490,491,492,493,494,495,496,497,498,499,500,
                 503,504,505,506,507,509,510,511,512,513,514]
super_title = 'H shot @ 1.5T'

In [23]:
angle_scan_ds, tilt_da = get_dataset_from_indices(indices_15T_H, anglescan_fl=True)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [26]:
plot_anglescan_IVs(angle_scan_ds, sup_title=super_title)
plot_anglescan_averaged_ts(angle_scan_ds, sup_title=super_title)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


---
### Low field Helium shot (0.8T, He)

In [19]:
super_title = 'He shot @ 0.8T'
indices_08T_He = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]

In [21]:
angle_scan_ds, tilt_da = get_dataset_from_indices(indices_08T_He, anglescan_fl=True, path_to_analysed_datasets=path_to_analysed_datasets)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [22]:
plot_anglescan_IVs(angle_scan_ds, sup_title=super_title)
plot_anglescan_averaged_ts(angle_scan_ds, sup_title=super_title)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


---
### Medium field Helium shot (1.2T, He)

In [23]:
indices_12T_He = [245, 246, 247, 248, 249, 250, 251, 252, 254, 255, 256, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271]
super_title = 'He shot @ 1.2T'

In [24]:
angle_scan_ds, tilt_da = get_dataset_from_indices(indices_12T_He, anglescan_fl=True, path_to_analysed_datasets=path_to_analysed_datasets)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [25]:
plot_anglescan_IVs(angle_scan_ds, sup_title=super_title)
plot_anglescan_averaged_ts(angle_scan_ds, sup_title=super_title)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


---
### Low field Deuterium shot (0.8T, D)

In [8]:
indices_08T_D = [462,463,464,465,466,467,468,469,470,471,472,475,476,477,478,479,480,481,482,483,484,485]
super_title = 'D shot @ 0.8T'

In [17]:
angle_scan_ds, tilt_da = get_dataset_from_indices(indices_08T_D, anglescan_fl=True)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [18]:
plot_anglescan_IVs(angle_scan_ds, sup_title=super_title)
plot_anglescan_averaged_ts(angle_scan_ds, sup_title=super_title)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


---
### Low Field Density Scan (0.8T, H)

In [12]:
indices_n_e_08T_H = [124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,142,143,144,145,146,147]
indices_n_e_12T_H = [199,200,201,202,203,204,205,206,207,208,209,210]
indices_n_e_08T_H_8deg = [276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,301,302,303,304,305,306,307,308,309,310,311]
indices_n_e_15T_H = [389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415]
super_title = 'H scan @ 0.8T'

In [13]:
dens_scan_ds, tilt_da = get_dataset_from_indices(indices_n_e_08T_H + indices_n_e_08T_H_8deg, anglescan_fl=False)
# dens_scan_ds, tilt_da = get_dataset_from_indices(indices_n_e_15T_H, anglescan_fl=False)
# get_dataset_metadata('analysed_2').reindex(indices_n_e_08T_H)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [14]:
dens_scan_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 56, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 119 120 121 122 ... 316 317 318
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 10.0 10.0 10.0 ... 8.0 8.0 8.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 8.066 ... 0.1196
    current                 (shot_number, probe, time) float64 0.3087 ... 0.2961
    shot_time               (shot_number, probe, time) float64 5.358 ... 8.106
    start_time              (shot_number, probe) float64 5.355 5.35 ... 8.103
    d_current               (shot_number, probe, time) float64 0.0005743 ... 0.001995
    ts_density              (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temp_max             (shot_number) float64 2.885 3.374 ... 0.9893 0.5413
    ts_dens_max             (shot_number) float64 1.88e+19 ... 4.147e+20
    ts_fl                   (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    shot_source_current     (shot_number) float64 99.9 100.2 ... 180.9 180.8
    shot_b_field            (shot_number) float64 0.8003 0.8003 ... 0.7999
    shot_tilt               (shot_number) float64 9.991 9.991 ... 7.988 7.988
    shot_target_pos         (shot_number) float64 -0.2 -0.2 -0.2 ... -0.2 -0.2
    shot_target_rot         (shot_number) float64 -89.94 -89.94 ... -89.94
    shot_target_pumpspeed   (shot_number) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_source_pumpspeed   (shot_number) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_heating_pumpspeed  (shot_number) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_source_pos         (shot_number) float64 1.35 1.35 1.35 ... 1.35 1.35
    shot_deuterium_gf       (shot_number) float64 0.0003817 ... 0.005159
    shot_hydrogen_gf        (shot_number) float64 8.444 8.445 ... 7.598 7.595
    shot_helium_gf          (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_hydrogen_puffed    (shot_number) float64 -0.01747 -0.0224 ... 8.444
    shot_helium_puffed      (shot_number) float64 -0.05099 -0.04909 ... -0.05513
    adc_timestamp           (shot_number) float64 6.698e+18 ... 6.699e+18
    adc_4_coax              (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_4_shunt_resistance  (shot_number) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_5_coax              (shot_number) float64 2.0 2.0 2.0 ... 2.0 2.0 2.0
    adc_5_shunt_resistance  (shot_number) float64 10.0 10.0 10.0 ... 1.0 1.0 1.0
    adc_calibration_fl      (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_fl                 (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    acquisition_length      (shot_number) float64 10.0 10.0 10.0 ... 22.5 22.5
    adc_end_timestamp       (shot_number) float64 6.698e+18 ... 6.699e+18
    adc_freqs               (shot_number) float64 1e+06 1e+06 ... 1e+06 1e+06

In [15]:
def plot_densscan_multi_ts(densscan_ds, ax=None, sup_title=None):
    ts_temp = densscan_ds['ts_temperature']
    ts_d_temp = densscan_ds['ts_d_temperature']
    ts_dens = densscan_ds['ts_density']
    ts_d_dens = densscan_ds['ts_d_density']
    
    if ax is None:
        fig, ax = plt.subplots(2)
    
    colourmap = plt.get_cmap('nipy_spectral')
    cycler = mpl.cycler(color=[colourmap(k) for k in np.linspace(0, 1, len(densscan_ds['shot_number'].values))])
    
    ax[0].set_title('Temperature')
    ax[0].set_prop_cycle(cycler)
    ax[1].set_title('Density')
    ax[1].set_prop_cycle(cycler)
    
    ts_temp.plot.line(hue='shot_number', ax=ax[0])
    ts_dens.plot.line(hue='shot_number', ax=ax[1])

    for i in [0, 1]:
        for probe_pos in [-6, 4, 14, 24]:
            ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
    plt.show()
    
    plt.suptitle(f'Thomson Scattering profiles for {sup_title}')
    
def plot_densscan_paramspace(densscan_ds, ax=None, sup_title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    densscan_ds.set_coords(['ts_temp_max', 'ts_dens_max']).plot.scatter(x='ts_temp_max', y='ts_dens_max', ax=ax)

    plt.show()    
    math_str = r'$T_e n_e$'
    plt.suptitle(f'Thomson Scattering {math_str} parameter space for {sup_title}')

In [16]:
plot_densscan_multi_ts(dens_scan_ds, sup_title=super_title)

In [17]:
plot_densscan_paramspace(dens_scan_ds)

In [18]:
# fig, ax = plt.subplots()
fit_data_df = fit_densscan_ds(dens_scan_ds, ax=True, threshold=0.01)

In [19]:
fit_data_df.columns

Index(['shot', 'probe', 'B', 'ts_temp', 'ts_dens', 'shot_time',
       'fit_success_fl', 'temp', 'd_temp', 'isat', 'd_isat', 'a', 'd_a', 'v_f',
       'd_v_f', 'dens', 'd_dens', 'chi2', 'reduced_chi2'],
      dtype='object')

In [58]:
fitter = fts.FullIVFitter()
colours = [['red', 'orange'], ['blue', 'purple']]

for index, row in fit_data_df.iterrows():
    i = index % 2
    if i == 0:
        fig, ax = plt.subplots()
    
    label = f'{row["probe"]}_{row["ts_dens"]:.2g}_{{}}'
    
    print(row['probe'], row['shot'])
    ds = dens_scan_ds.sel(shot_number=row['shot'], probe=row['probe']).max('ts_radial_pos')
    ds = ds.where(ds.current < 0.01, drop=True)
    
#     (-ds.set_coords('voltage')['current']).plot.line(x='voltage', ax=ax, label=label.format('data'))
    ax.errorbar(ds['voltage'], -ds['current'].values, yerr=ds['d_current'].values, 
                label=label.format('data'), ecolor='silver', color=colours[i][0])
    
    fit_params = [
        row['isat'],
        row['a'],
        row['temp'],
        row['v_f']
    ]
    fit_reconstruction = fitter.fit_function(ds['voltage'].values, *fit_params)
    ax.plot(ds['voltage'].values, fit_reconstruction, label=label.format('fit'), color=colours[i][1])  
    ax.legend()
        

L 119
S 119
L 120
S 120
L 121
S 121
L 122
S 122
L 123
S 123
L 124
S 124
L 125
S 125
L 126
S 126
L 127
S 127
L 128
S 128
L 129
S 129
L 130
S 130
L 131
S 131
L 132
S 132
L 133
S 133
L 134
S 134
L 135
S 135
L 137
S 137
L 138
S 138


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


L 139
S 139
L 140


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


S 140
L 141
S 141


...

S 314
L 315
S 315


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


L 316
S 316
L 317
S 317


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


L 318
S 318


In [20]:
ds = fit_data_df.to_xarray().set_coords('shot').swap_dims({'index':'shot'})
s_shots = ds.where(ds.probe=='S', drop=True)
l_shots = ds.where(ds.probe=='L', drop=True)
probes_da = xr.DataArray(['S', 'L'], dims=['probe'], name='probe')

ds_stacked = xr.concat([l_shots.drop('probe'), s_shots.drop('probe')], dim=probes_da).drop('index')
print(ds_stacked)
# print()
# print(dens_scan_ds)

<xarray.Dataset>
Dimensions:         (probe: 2, shot: 56)
Coordinates:
  * shot            (shot) object 119 120 121 122 123 ... 314 315 316 317 318
  * probe           (probe) <U1 'S' 'L'
Data variables:
    B               (probe, shot) float64 0.8 0.8 0.8 0.8 ... 0.8 0.8 0.8 0.8
    ts_temp         (probe, shot) object 2.8846944816983706 ... 0.5413477056504756
    ts_dens         (probe, shot) object 1.879597755399353e+19 ... 4.147405358873392e+20
    shot_time       (probe, shot) datetime64[ns] 2019-06-03T12:25:56.459635 ... 2019-06-05T13:07:29.730126
    fit_success_fl  (probe, shot) object True True True True ... True True True
    temp            (probe, shot) float64 2.454 2.362 3.515 ... 0.9417 0.9535
    d_temp          (probe, shot) float64 0.07237 0.08255 ... 0.005656 0.01655
    isat            (probe, shot) float64 0.005614 0.005767 ... 0.6263 0.225
    d_isat          (probe, shot) float64 5.505e-05 6.19e-05 ... 0.0004643
    a               (probe, shot) float64 0.09845

In [21]:
fig, ax = plt.subplots()
ds_stacked.set_coords(['temp', 'dens']).plot.scatter(x='temp', y='dens', hue='probe', markersize='shot', ax=ax)
plot_densscan_paramspace(dens_scan_ds, ax=ax)
# ax.errorbar(ds_stacked.sel(probe='S')[])

In [22]:
fig, ax = plt.subplots(2)

ds_stacked.plot.scatter(x='shot', y='temp', hue='probe', hue_style='discrete', ax=ax[0])
dens_scan_ds.max('ts_radial_pos').plot.scatter(x='shot_number', y='ts_temp_max', hue='probe', hue_style='discrete', ax=ax[0])

ds_stacked.plot.scatter(x='shot', y='dens', hue='probe', hue_style='discrete', ax=ax[1])
dens_scan_ds.max('ts_radial_pos').plot.scatter(x='shot_number', y='ts_dens_max', hue='probe', hue_style='discrete', ax=ax[1])


In [23]:
fig, ax = plt.subplots(2)

ax[0].plot(s_shots['shot'], s_shots['temp'] / s_shots['ts_temp'], label='S')
ax[0].plot(l_shots['shot'], l_shots['temp'] / l_shots['ts_temp'], label='L')
ax[0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[0].legend()

ax[1].plot(s_shots['shot'], s_shots['dens'] / s_shots['ts_dens'], label='S')
ax[1].plot(l_shots['shot'], l_shots['dens'] / l_shots['ts_dens'], label='L')
ax[1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[1].legend()

In [24]:
# Plot the ratio of the ts measured vals and the probe measured vals

fig, ax = plt.subplots(2)

ax[0].plot(s_shots['ts_temp'], s_shots['temp'] / s_shots['ts_temp'], label='S', marker='x', linestyle='none')
ax[0].plot(l_shots['ts_temp'], l_shots['temp'] / l_shots['ts_temp'], label='L', marker='x', linestyle='none')
ax[0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[0].legend()

ax[1].plot(s_shots['ts_dens'], s_shots['dens'] / s_shots['ts_dens'], label='S', marker='x', linestyle='none')
ax[1].plot(l_shots['ts_dens'], l_shots['dens'] / l_shots['ts_dens'], label='L', marker='x', linestyle='none')
ax[1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[1].legend()

In [25]:
fig, ax = plt.subplots(2)

ax[0].plot(s_shots['ts_temp'], (s_shots['temp'] / s_shots['ts_temp']) - (l_shots['temp'] / l_shots['ts_temp']), label='diff')
# ax[0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[0].legend()

ax[1].plot(s_shots['ts_dens'], (s_shots['dens'] / s_shots['ts_dens']) - (l_shots['dens'] / l_shots['ts_dens']), label='diff')
# ax[1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[1].legend()

In [26]:
fig, ax = plt.subplots(2)

ax[0].errorbar(s_shots['shot'], s_shots['temp'], yerr=s_shots['d_temp'], label='S')
ax[0].errorbar(l_shots['shot'], l_shots['temp'], yerr=l_shots['d_temp'], label='L')
ax[0].errorbar(s_shots['shot'], s_shots['ts_temp'], yerr=dens_scan_ds.max(['ts_radial_pos','time']).sel(probe='S')['ts_d_temperature'].values, 
               color='tab:green', ecolor='tab:green', label='Thomson')
ax[0].axhline(y=0, **c.AX_LINE_DEFAULTS)
ax[0].legend()

ax[1].errorbar(s_shots['shot'], s_shots['dens'], yerr=s_shots['d_dens'], label='S')
ax[1].errorbar(l_shots['shot'], l_shots['dens'], yerr=l_shots['d_dens'], label='L')
ax[1].errorbar(s_shots['shot'], s_shots['ts_dens'], yerr=dens_scan_ds.max(['ts_radial_pos','time']).sel(probe='S')['ts_d_density'].values, 
               color='tab:green', ecolor='tab:green', label='Thomson')
ax[1].axhline(y=0, **c.AX_LINE_DEFAULTS)
ax[1].legend()

In [27]:
s_shots = s_shots.sortby('ts_dens')
l_shots = l_shots.sortby('ts_dens')
s_shots

<xarray.Dataset>
Dimensions:         (shot: 56)
Coordinates:
    index           (shot) int64 47 91 93 17 49 15 19 ... 111 85 41 87 109 89 39
  * shot            (shot) object 283 308 309 127 284 ... 140 303 317 304 139
Data variables:
    probe           (shot) object 'S' 'S' 'S' 'S' 'S' ... 'S' 'S' 'S' 'S' 'S'
    B               (shot) float64 0.8 0.8 0.8 0.8 0.8 ... 0.8 0.8 0.8 0.8 0.8
    ts_temp         (shot) object 2.3658216933436154 ... 0.7965493228324931
    ts_dens         (shot) object 1.1910288184069587e+19 ... 4.6671517181193853e+20
    shot_time       (shot) datetime64[ns] 2019-06-05T10:17:02.733302 ... 2019-06-03T14:09:47.130215
    fit_success_fl  (shot) object True True True True ... True True True True
    temp            (shot) float64 3.499 3.281 3.199 ... 0.9417 0.9889 6.357
    d_temp          (shot) float64 0.03731 0.02325 0.01614 ... 0.01408 0.2342
    isat            (shot) float64 0.09011 0.07975 0.07881 ... 0.6032 0.4515
    d_isat          (shot) float64 0.0001928 0.0001056 ... 0.001789 0.007777
    a               (shot) float64 0.02415 0.02821 0.02862 ... 0.005443 0.02413
    d_a             (shot) float64 0.0003186 0.0001946 ... 0.0001634 0.003325
    v_f             (shot) float64 -31.32 -29.99 -29.78 ... -3.124 -3.124 -2.866
    d_v_f           (shot) float64 0.02885 0.01789 0.0091 ... 0.004796 0.07481
    dens            (shot) float64 9.456e+18 8.641e+18 ... 1.191e+20 2.925e+19
    d_dens          (shot) float64 7.647e+17 6.978e+17 ... 9.648e+18 2.088e+18
    chi2            (shot) float64 88.26 9.788 10.64 182.4 ... 14.36 388.2 475.4
    reduced_chi2    (shot) float64 3.044 0.3375 0.3547 ... 0.334 9.028 11.88

In [28]:
fig, ax = plt.subplots(2, 2)

ax[0][0].errorbar(s_shots['ts_dens'], s_shots['temp'], yerr=s_shots['d_temp'], label='S', linestyle='none', marker='x')
ax[0][0].errorbar(l_shots['ts_dens'], l_shots['temp'], yerr=l_shots['d_temp'], label='L', linestyle='none', marker='x')
ax[0][0].errorbar(s_shots['ts_dens'], s_shots['ts_temp'], yerr=dens_scan_ds.max(['ts_radial_pos','time']).sel(probe='S')['ts_d_temperature'].values, 
               color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker='x')
ax[0][0].set_xscale('log')
ax[0][0].legend()

ax[0][1].errorbar(s_shots['ts_dens'], s_shots['dens'], yerr=s_shots['d_dens'], label='S', linestyle='none', marker='x')
ax[0][1].errorbar(l_shots['ts_dens'], l_shots['dens'], yerr=l_shots['d_dens'], label='L', linestyle='none', marker='x')
ax[0][1].errorbar(s_shots['ts_dens'], s_shots['ts_dens'], yerr=dens_scan_ds.max(['ts_radial_pos','time']).sel(probe='S')['ts_d_density'].values, 
                  color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker='x')
ax[0][1].set_yscale('log')
ax[0][1].set_xscale('log')
ax[0][1].legend()

ax[1][0].plot(s_shots['ts_dens'], s_shots['temp'] / s_shots['ts_temp'], label='S', linestyle='none', marker='x')
ax[1][0].plot(l_shots['ts_dens'], l_shots['temp'] / l_shots['ts_temp'], label='L', linestyle='none', marker='x')
ax[1][0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[1][0].set_xscale('log')
ax[1][0].legend()

ax[1][1].plot(s_shots['ts_dens'], s_shots['dens'] / s_shots['ts_dens'], label='S', linestyle='none', marker='x')
ax[1][1].plot(l_shots['ts_dens'], l_shots['dens'] / l_shots['ts_dens'], label='L', linestyle='none', marker='x')
ax[1][1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[1][1].set_xscale('log')
ax[1][1].legend()

In [29]:
colours = [['red', 'orange'], ['blue', 'purple']]

for index, row in fit_data_df.iterrows():
    ds = dens_scan_ds.sel(shot_number=row['shot'], probe=row['probe'])
    label = f'{row["probe"]}_{row["ts_dens"]:.2g}_{{}}'
    
    i = index % 2
    if i == 0:
        fig, ax = plt.subplots(2)
        
        ax[0].set_title('Temperature')
        ax[1].set_title('Density')
    
        ds['ts_temperature'].plot.line(x='ts_radial_pos', ax=ax[0], label=f'{row["ts_dens"]:.2g}')
        ds['ts_density'].plot.line(x='ts_radial_pos', ax=ax[1], label=f'{row["ts_dens"]:.2g}')
    
    ax[0].axhline(y=row['temp'], color=colours[i][0], linewidth=1, linestyle='dashed', label=label.format('probes'))
    ax[0].axhspan(row['temp'] + row['d_temp'], row['temp'] - row['d_temp'], color=colours[i][1], alpha=0.25)

    ax[1].axhline(y=row['dens'], color=colours[i][0], linewidth=1, linestyle='dashed', label=label.format('probes'))
    ax[1].axhspan(row['dens'] + row['d_dens'], row['dens'] - row['d_dens'], color=colours[i][1], alpha=0.25)
    
    for i in [0, 1]:
        for probe_pos in [-6, 4, 14, 24]:
            ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
    plt.show()
    
    ax[0].legend()
    ax[1].legend()

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


In [30]:
# Plot the average TS profile
fig, ax = plt.subplots(2)
ts_ds = dens_scan_ds[['ts_temperature', 'ts_density']].mean('shot_number')
ts_ds['ts_temperature'].plot.line(ax=ax[0])
ts_ds['ts_density'].plot.line(ax=ax[1])

for i in [0, 1]:
    for probe_pos in [-6, 4, 14, 24]:
        ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
plt.show()

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [31]:
# s_shots
print(len(indices_n_e_08T_H_8deg))
print(indices_n_e_08T_H_8deg)
firstscan_shot_numbers = dens_scan_ds.swap_dims({'shot_number':'adc_index'}).sel(adc_index=indices_n_e_08T_H_8deg)
secondscan_shot_numbers = dens_scan_ds.swap_dims({'shot_number':'adc_index'}).sel(adc_index=indices_n_e_08T_H)
secondscan_shot_numbers.tilt.mean()

33
[276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311]


<xarray.DataArray 'tilt' ()>
array(10.)

In [32]:
magnum_probes['S'].get_collection_area()

TypeError: get_collection_area() missing 1 required positional argument: 'alpha'

In [33]:
# Plot comparing the density scans of hydrogen at 0.8T on the S and L probes at two slightly different tilts

fig, ax = plt.subplots(2, 2, sharex=True)

markers = ['x', '^']
scan_indices = [indices_n_e_08T_H, indices_n_e_08T_H_8deg]
# scan_indices = [indices_n_e_15T_H, indices_n_e_15T_H]
labels = [r' - 10$^{\circ}$ tilt', r' - 8$^{\circ}$ tilt']

for i in range(2):
    ts_scan = dens_scan_ds.swap_dims({'shot_number':'adc_index'}).sel(adc_index=scan_indices[i]).max('ts_radial_pos')
    s_shots_scan = s_shots.sel(shot=ts_scan['shot_number'])
    l_shots_scan = l_shots.sel(shot=ts_scan['shot_number'])
    
    ax[0][0].errorbar(s_shots_scan['ts_dens'], s_shots_scan['temp'], yerr=s_shots_scan['d_temp'], label='S'+labels[i],
                      color='tab:blue', ecolor='tab:blue', linestyle='none', marker=markers[i])
    ax[0][0].errorbar(l_shots_scan['ts_dens'], l_shots_scan['temp'], yerr=l_shots_scan['d_temp'], label='L'+labels[i], 
                      color='tab:orange', ecolor='tab:orange', linestyle='none', marker=markers[i])
    ax[0][0].errorbar(s_shots_scan['ts_dens'], s_shots_scan['ts_temp'], yerr=ts_scan['ts_d_temperature'].values, 
                      color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker=markers[i])
    ax[0][0].set_xscale('log')
    ax[0][0].legend()

    ax[0][1].errorbar(s_shots_scan['ts_dens'], s_shots_scan['dens'], yerr=s_shots_scan['d_dens'], label='S'+labels[i],
                      color='tab:blue', ecolor='tab:blue', linestyle='none', marker=markers[i])
    ax[0][1].errorbar(l_shots_scan['ts_dens'], l_shots_scan['dens'], yerr=l_shots_scan['d_dens'], label='L'+labels[i], 
                      color='tab:orange', ecolor='tab:orange', linestyle='none', marker=markers[i])
    ax[0][1].errorbar(s_shots_scan['ts_dens'], s_shots_scan['ts_dens'], yerr=ts_scan['ts_d_density'].values, 
                      color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker=markers[i])
    ax[0][1].set_yscale('log')
    ax[0][1].set_xscale('log')
    ax[0][1].legend()

    ax[1][0].plot(s_shots_scan['ts_dens'], s_shots_scan['temp'] / s_shots_scan['ts_temp'], label='S'+labels[i], 
                  color='tab:blue', linestyle='none', marker=markers[i])
    ax[1][0].plot(l_shots_scan['ts_dens'], l_shots_scan['temp'] / l_shots_scan['ts_temp'], label='L'+labels[i], 
                  color='tab:orange', linestyle='none', marker=markers[i])
    ax[1][0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][0].set_xscale('log')
    ax[1][0].legend()

    ax[1][1].plot(s_shots_scan['ts_dens'], s_shots_scan['dens'] / s_shots_scan['ts_dens'], label='S'+labels[i], 
                  color='tab:blue', linestyle='none', marker=markers[i])
    ax[1][1].plot(l_shots_scan['ts_dens'], l_shots_scan['dens'] / l_shots_scan['ts_dens'], label='L'+labels[i], 
                  color='tab:orange', linestyle='none', marker=markers[i])
    ax[1][1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][1].set_xscale('log')
    ax[1][1].legend()
    
fig.suptitle('Plots of temperature and density measurements during the density scan at 0.8T in hydrogen')
ax[0][0].set_title('Temperature')
ax[0][0].set_ylabel(r'$T_e$ (eV)')
ax[0][1].set_title('Density')
ax[0][1].set_ylabel(r'$n_e$ (m$^{-3}$)')

ax[1][0].set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
ax[1][0].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1][1].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1][1].set_ylabel(r'$n_{e,probe}$ / $n_{e,TS}$')

Text(0, 0.5, '$n_{e,probe}$ / $n_{e,TS}$')

In [50]:
# Plot comparing the density scans of hydrogen at 0.8T on the S and L probes at two slightly different tilts

fig, ax = plt.subplots(2, 2, sharex=True)

markers = ['x', '^']
scan_indices = [indices_n_e_08T_H, indices_n_e_08T_H_8deg]
labels = [r' - 10$^{\circ}$ tilt', r' - 8$^{\circ}$ tilt']

for i in range(2):
    ts_scan = dens_scan_ds.swap_dims({'shot_number':'adc_index'}).sel(adc_index=scan_indices[i]).max('ts_radial_pos')
    s_shots_scan = s_shots.sel(shot=ts_scan['shot_number'])
    l_shots_scan = l_shots.sel(shot=ts_scan['shot_number'])
    
    ax[0][0].errorbar(s_shots_scan['ts_temp'], s_shots_scan['temp'], yerr=s_shots_scan['d_temp'], label='S'+labels[i],
                      color='tab:blue', ecolor='tab:blue', linestyle='none', marker=markers[i])
    ax[0][0].errorbar(l_shots_scan['ts_temp'], l_shots_scan['temp'], yerr=l_shots_scan['d_temp'], label='L'+labels[i], 
                      color='tab:orange', ecolor='tab:orange', linestyle='none', marker=markers[i])
    ax[0][0].errorbar(s_shots_scan['ts_temp'], s_shots_scan['ts_temp'], yerr=ts_scan['ts_d_temperature'].values, 
                      color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker=markers[i])
#     ax[0][0].set_xscale('log')
    ax[0][0].legend()

    ax[0][1].errorbar(s_shots_scan['ts_temp'], s_shots_scan['dens'], yerr=s_shots_scan['d_dens'], label='S'+labels[i],
                      color='tab:blue', ecolor='tab:blue', linestyle='none', marker=markers[i])
    ax[0][1].errorbar(l_shots_scan['ts_temp'], l_shots_scan['dens'], yerr=l_shots_scan['d_dens'], label='L'+labels[i], 
                      color='tab:orange', ecolor='tab:orange', linestyle='none', marker=markers[i])
    ax[0][1].errorbar(s_shots_scan['ts_temp'], s_shots_scan['ts_dens'], yerr=ts_scan['ts_d_density'].values, 
                      color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker=markers[i])
    ax[0][1].set_yscale('log')
#     ax[0][1].set_xscale('log')
    ax[0][1].legend()

    ax[1][0].plot(s_shots_scan['ts_temp'], s_shots_scan['temp'] / s_shots_scan['ts_temp'], label='S'+labels[i], 
                  color='tab:blue', linestyle='none', marker=markers[i])
    ax[1][0].plot(l_shots_scan['ts_temp'], l_shots_scan['temp'] / l_shots_scan['ts_temp'], label='L'+labels[i], 
                  color='tab:orange', linestyle='none', marker=markers[i])
    ax[1][0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
#     ax[1][0].set_xscale('log')
    ax[1][0].legend()

    ax[1][1].plot(s_shots_scan['ts_temp'], s_shots_scan['dens'] / s_shots_scan['ts_dens'], label='S'+labels[i], 
                  color='tab:blue', linestyle='none', marker=markers[i])
    ax[1][1].plot(l_shots_scan['ts_temp'], l_shots_scan['dens'] / l_shots_scan['ts_dens'], label='L'+labels[i], 
                  color='tab:orange', linestyle='none', marker=markers[i])
    ax[1][1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
#     ax[1][1].set_xscale('log')
    ax[1][1].legend()
    
fig.suptitle('Plots of temperature and density measurements during the density scan at 0.8T in hydrogen')
ax[0][0].set_title('Temperature')
ax[0][0].set_ylabel(r'$T_e$ (eV)')
ax[0][1].set_title('Density')
ax[0][1].set_ylabel(r'$n_e$ (m$^{-3}$)')

ax[1][0].set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
ax[1][0].set_xlabel(r'$T_{e,TS}$ (eV)')
ax[1][1].set_xlabel(r'$T_{e,TS}$ (eV)')
ax[1][1].set_ylabel(r'$n_{e,probe}$ / $n_{e,TS}$')

Text(0, 0.5, '$n_{e,probe}$ / $n_{e,TS}$')

In [56]:
for i in range(2):
    ts_scan = dens_scan_ds.swap_dims({'shot_number':'adc_index'}).sel(adc_index=scan_indices[i]).max('ts_radial_pos')
    s_shots_scan = s_shots.sel(shot=ts_scan['shot_number'])
    l_shots_scan = l_shots.sel(shot=ts_scan['shot_number'])
    
    s_q_par = (s_shots_scan['isat'] / magnum_probes['S'].get_collection_area(s_shots_scan['tilt'])) * 7 * s_shots_scan['temp']
    l_q_par = (l_shots_scan['isat'] / magnum_probes['L'].get_collection_area(l_shots_scan['tilt'])) * 7 * l_shots_scan['temp']
    ts_q_par = (lp.sound_speed(s_shots_scan['ts_temp'].astype(np.float64)) * s_shots_scan['ts_dens'].values * 1.6e-19) * 7 * s_shots_scan['ts_temp'].values
    

<xarray.DataArray 'ts_temp' (adc_index: 23)>
array([2.88469448, 3.37384515, 1.85990914, 2.6772085 , 2.62357246,
       1.15747117, 0.99904936, 3.20695985, 3.0018972 , 2.96452242,
       2.54145309, 1.91857402, 1.11447433, 0.88808922, 0.76344101,
       0.6472517 , 0.66011424, 0.70000733, 0.78039967, 0.79654932,
       0.80041146, 0.81191461, 0.78467977])
Coordinates:
    index         (adc_index) int64 1 3 5 7 9 11 13 15 ... 33 35 37 39 41 43 45
    shot          (adc_index) object 119 120 121 122 123 ... 138 139 140 141 142
    shot_number   (adc_index) int32 119 120 121 122 123 ... 138 139 140 141 142
    ts_number     (adc_index) float64 ...
    ts_timestamp  (adc_index) float64 ...
    ts_time       (adc_index) datetime64[ns] ...
  * adc_index     (adc_index) float64 124.0 125.0 126.0 ... 145.0 146.0 147.0
    adc_time      (adc_index) datetime64[ns] ...
    tilt          (adc_index) float64 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0
<xarray.DataArray 'ts_temp' (adc_index: 33)>
array([

In [57]:
# Plot comparing the density scans of hydrogen at 0.8T on the S and L probes at two slightly different tilts

fig, ax = plt.subplots(2, 2, sharex=True)

markers = ['x', '^']
scan_indices = [indices_n_e_08T_H, indices_n_e_08T_H_8deg]
# scan_indices = [indices_n_e_15T_H, indices_n_e_15T_H]
labels = [r' - 10$^{\circ}$ tilt', r' - 8$^{\circ}$ tilt']

for i in range(2):
    ts_scan = dens_scan_ds.swap_dims({'shot_number':'adc_index'}).sel(adc_index=scan_indices[i]).max('ts_radial_pos')
    s_shots_scan = s_shots.sel(shot=ts_scan['shot_number'])
    l_shots_scan = l_shots.sel(shot=ts_scan['shot_number'])
    
    s_q_par = (s_shots_scan['isat'] / magnum_probes['S'].get_collection_area(s_shots_scan['tilt'])) * 7 * s_shots_scan['temp']
    l_q_par = (l_shots_scan['isat'] / magnum_probes['L'].get_collection_area(l_shots_scan['tilt'])) * 7 * l_shots_scan['temp']
    ts_q_par = (lp.sound_speed(s_shots_scan['ts_temp'].astype(np.float64)) * s_shots_scan['ts_dens'].values * 1.6e-19) * 7 * s_shots_scan['ts_temp'].values
    
    ax[0][0].errorbar(s_shots_scan['ts_dens'], s_q_par, yerr=s_shots_scan['d_temp'], label='S'+labels[i],
                      color='tab:blue', ecolor='tab:blue', linestyle='none', marker=markers[i])
    ax[0][0].errorbar(t_shots_scan['ts_dens'], l_shots_scan['temp'], yerr=l_shots_scan['d_temp'], label='L'+labels[i], 
                      color='tab:orange', ecolor='tab:orange', linestyle='none', marker=markers[i])
    ax[0][0].errorbar(s_shots_scan['ts_dens'], s_shots_scan['ts_temp'], yerr=ts_scan['ts_d_temperature'].values, 
                      color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker=markers[i])
    ax[0][0].set_xscale('log')
    ax[0][0].legend()

    ax[0][1].errorbar(ts_q_par, s_shots_scan['dens'], yerr=s_shots_scan['d_dens'], label='S'+labels[i],
                      color='tab:blue', ecolor='tab:blue', linestyle='none', marker=markers[i])
    ax[0][1].errorbar(ts_q_par, l_shots_scan['dens'], yerr=l_shots_scan['d_dens'], label='L'+labels[i], 
                      color='tab:orange', ecolor='tab:orange', linestyle='none', marker=markers[i])
    ax[0][1].errorbar(ts_q_par, s_shots_scan['ts_dens'], yerr=ts_scan['ts_d_density'].values, 
                      color='tab:green', ecolor='tab:green', label='Thomson', linestyle='none', marker=markers[i])
    ax[0][1].set_yscale('log')
    ax[0][1].set_xscale('log')
    ax[0][1].legend()

    ax[1][0].plot(s_shots_scan['ts_dens'], s_q_par / s_shots_scan['ts_temp'], label='S'+labels[i], 
                  color='tab:blue', linestyle='none', marker=markers[i])
    ax[1][0].plot(s_shots_scan['ts_dens'], l_shots_scan['temp'] / l_shots_scan['ts_temp'], label='L'+labels[i], 
                  color='tab:orange', linestyle='none', marker=markers[i])
    ax[1][0].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][0].set_xscale('log')
    ax[1][0].legend()

    ax[1][1].plot(ts_q_par, s_shots_scan['dens'] / s_shots_scan['ts_dens'], label='S'+labels[i], 
                  color='tab:blue', linestyle='none', marker=markers[i])
    ax[1][1].plot(ts_q_par, l_shots_scan['dens'] / l_shots_scan['ts_dens'], label='L'+labels[i], 
                  color='tab:orange', linestyle='none', marker=markers[i])
    ax[1][1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
    ax[1][1].set_xscale('log')
    ax[1][1].legend()
    
fig.suptitle('Plots of temperature and density measurements during the density scan at 0.8T in hydrogen')
ax[0][0].set_title('Temperature')
ax[0][0].set_ylabel(r'$T_e$ (eV)')
ax[0][1].set_title('Density')
ax[0][1].set_ylabel(r'$n_e$ (m$^{-3}$)')

ax[1][0].set_ylabel(r'$T_{e,probe}$ / $T_{e,TS}$')
ax[1][0].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1][1].set_xlabel(r'$n_{e,TS}$ (m$^{-3}$)')
ax[1][1].set_ylabel(r'$n_{e,probe}$ / $n_{e,TS}$')

Text(0, 0.5, '$n_{e,probe}$ / $n_{e,TS}$')

---
### Axial Position Scan (0.8T, H)

In [32]:
ts_calibration_indices = [122,124,125,131,132] 
# 162 taken out as it seems to have different length...
axial_scan_indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

In [33]:
axial_scan_ds, tilt_da = get_dataset_from_indices(axial_scan_indices, anglescan_fl=False)
# get_dataset_metadata('analysed_2').reindex(indices_n_e_08T_H)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [34]:
axial_scan_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 42, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 127 143 144 145 ... 182 183 184
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 10.0 10.0 10.0 ... 10.0 10.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 8.064 ... -94.66
    current                 (shot_number, probe, time) float64 0.3151 ... -0.01196
    shot_time               (shot_number, probe, time) float64 5.359 ... 4.259
    start_time 

In [35]:
fit_data_df = fit_anglescan_ds(axial_scan_ds, ax=True, threshold=2, scan_param='shot_number', probes=('S', 'L'))

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)

In [36]:
ds = fit_data_df.to_xarray()

ds = ds.set_coords(['probe', 'shot_number']).swap_dims({'index':'shot_number'}).drop('index')
s_shots = ds.where(ds.probe=='S', drop=True).drop('probe')
l_shots = ds.where(ds.probe=='L', drop=True).drop('probe')

probes_da = xr.DataArray(['S', 'L'], dims=['probe'], name='probe')

ds_stacked = xr.concat([s_shots, l_shots], dim=probes_da)
ds_stacked

<xarray.Dataset>
Dimensions:         (probe: 2, shot_number: 42)
Coordinates:
  * shot_number     (shot_number) object 127 143 144 145 146 ... 181 182 183 184
  * probe           (probe) <U1 'S' 'L'
Data variables:
    B               (probe, shot_number) float64 0.8 0.8 0.8 0.8 ... 0.8 0.8 0.8
    ts_temp         (probe, shot_number) object 3.0018971964737067 nan ... nan
    ts_dens         (probe, shot_number) object 1.5935223487373072e+19 ... nan
    fit_success_fl  (probe, shot_number) object True True True ... True True
    temp            (probe, shot_number) float64 3.528 4.16 ... 2.767 2.462
    d_temp          (probe, shot_number) float64 0.05943 0.1042 ... 0.07122
    isat            (probe, shot_number) float64 0.07176 0.07042 ... 0.005463
    d_isat          (probe, shot_number) float64 0.0003856 ... 8.084e-05
    a               (probe, shot_number) float64 0.02247 0.02127 ... 0.09014
    d_a             (probe, shot_number) float64 0.0008051 0.003076 ... 0.002536
    v_f 

In [37]:
ds_combined = xr.merge([axial_scan_ds, ds_stacked])
ds_combined

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 42, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 127 143 144 145 ... 182 183 184
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 10.0 10.0 10.0 ... 10.0 10.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 8.064 ... -94.66
    current                 (shot_number, probe, time) float64 0.3151 ... -0.01196
    shot_time               (shot_number, probe, time) float64 5.359 ... 4.259
    start_time 

In [38]:
# TS Profiles with probe positions overlaid
fig, ax = plt.subplots(2, sharex=True)
ds_combined.sel(shot_number=127)['ts_temperature'].plot.line(hue='shot_number', ax=ax[0])
ds_combined.sel(shot_number=127)['ts_density'].plot.line(hue='shot_number', ax=ax[1])

colours = {
    'L': 'blue',
    'S': 'red',
    'B': 'green',
    'R': 'gold'
}
for probe, pos in magnum_probes.probe_position.items():
    probe = probe.upper()
    ax[0].axvline(x=pos, linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[1].axvline(x=pos, linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
ax[0].legend()
ax[1].legend()

In [1]:
# Temperature and Density measurements as a function of axis
fig, ax = plt.subplots(3)

for probe in ds_combined.probe.values:
    ds = ds_combined.sel(probe=probe).sortby('shot_target_pos')
    ax[0].errorbar('shot_target_pos', 'temp', yerr='d_temp', data=ds, label=probe, linestyle='none', marker='.')
    ax[1].errorbar('shot_target_pos', 'dens', yerr='d_dens', data=ds, label=probe, linestyle='none', marker='.')
    ax[2].plot('shot_target_pos', 'reduced_chi2', data=ds, label=probe)
    
ax[0].legend()
ax[1].legend()
ax[0].axhline(y=ds_combined['ts_temp'].values[0][0], **c.AX_LINE_DEFAULTS)
ax[1].axhline(y=ds_combined['ts_dens'].values[0][0], **c.AX_LINE_DEFAULTS)
ax[2].axhline(y=1.0, **c.AX_LINE_DEFAULTS)

NameError: name 'plt' is not defined

In [40]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row')

colours = {
    'L': 'blue',
    'S': 'red',
    'B': 'green',
    'R': 'gold'
}

ts_ds = ds_combined.sel(shot_number=127)
ts_ds['ts_temperature'].plot.line(hue='shot_number', ax=ax[0][0])
ts_ds['ts_density'].plot.line(hue='shot_number', ax=ax[1][0])

for probe in ds_combined.probe.values:
    ds = ds_combined.sel(probe=probe).sortby('shot_target_pos')
    ax[0][1].errorbar('shot_target_pos', 'temp', yerr='d_temp', data=ds, label=probe, linestyle='none', marker='.', color=colours[probe])
    ax[1][1].errorbar('shot_target_pos', 'dens', yerr='d_dens', data=ds, label=probe, linestyle='none', marker='.', color=colours[probe])

    ax[0][0].axvline(x=magnum_probes.probe_position[probe.lower()], linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[1][0].axvline(x=magnum_probes.probe_position[probe.lower()], linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[0][1].axhline(y=ts_ds.ts_temperature.interp(ts_radial_pos=magnum_probes.probe_position[probe.lower()]).values, 
                     linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[0][1].axhline(y=ts_ds.ts_temperature.interp(ts_radial_pos=magnum_probes.probe_position[probe.lower()]).values, 
                     linestyle='dashed', linewidth=0.75, color=colours[probe], label=probe)
    ax[1][1].axhline(y=ts_ds.ts_density.interp(ts_radial_pos=magnum_probes.probe_position[probe.lower()]).values, 
                     linestyle='dashed', linewidth=0.75, color=colours[probe], label='TS '+probe)
    
ax[0][1].axhline(y=ds_combined['ts_temp'].values[0][0], **c.AX_LINE_DEFAULTS, label='TS peak')
ax[1][1].axhline(y=ds_combined['ts_dens'].values[0][0], **c.AX_LINE_DEFAULTS, label='TS peak')
ax[1][0].legend()
ax[1][1].legend()
ax[0][0].legend()
ax[0][1].legend()

---
### Single Bias Shots FFT Analysis (1.2T, H)

In [40]:
single_bias_indices = [116,117]

In [79]:
single_bias_ds, tilt_da = get_dataset_from_indices(single_bias_indices, path_to_analysed_datasets='analysed_2', anglescan_fl=False, preprocess='sel')
# get_dataset_metadata('analysed_2').reindex(indices_n_e_08T_H)

<xarray.DataArray 'time' (time: 250)>
array([0.    , 0.0001, 0.0002, ..., 0.0247, 0.0248, 0.0249])
Coordinates:
  * time     (time) float64 0.0 0.0001 0.0002 0.0003 ... 0.0247 0.0248 0.0249
<xarray.DataArray 'time' (time: 250)>
array([0.    , 0.0001, 0.0002, ..., 0.0247, 0.0248, 0.0249])
Coordinates:
  * time     (time) float64 0.0 0.0001 0.0002 0.0003 ... 0.0247 0.0248 0.0249


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [75]:
single_bias_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 2, sweep: 189, time: 250, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 111 112
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 10.0 0.0
Dimensions without coordinates: sweep, time
Data variables:
    voltage                 (shot_number, probe, sweep, time) float64 -101.6 ... -102.2
    current                 (shot_number, probe, sweep, time) float64 -0.1047 ... -0.001299
    shot_time               (shot_number, probe, sweep, time) float64 1.117 ... 10.77
    start_time  

In [58]:
for shot in single_bias_ds.shot_number.values:
    ds = single_bias_ds.sel(shot_number=shot).max('ts_radial_pos')
    
    fig, axes = plt.subplots(len(ds.probe.values))
    
    for i, probe in enumerate(ds.probe.values):
        ds.sel(probe=probe).set_coords('shot_time')['current'].plot.line(x='shot_time', hue='sweep', ax=axes[i])
        axes[i].set_title(f'{probe} - tilt={ds.tilt.values}')

In [82]:
single_bias_ds = single_bias_ds.stack({'proper_time': ['sweep', 'time']}).reset_index('proper_time')

single_bias_ds['proper_time'] = single_bias_ds['proper_time'] * 0.0001

In [84]:
single_bias_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, proper_time: 47250, shot_number: 2, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 111 112
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 10.0 0.0
    sweep                   (proper_time) int64 0 0 0 0 0 ... 188 188 188 188
    time                    (proper_time) int64 0 1 2 3 4 ... 246 247 248 249
  * proper_time             (proper_time) float64 0.0 0.0001 ... 4.725 4.725
Data variables:
    voltage                 (shot_number, probe, proper_time) float64 -101.6 ... -102.2
    curr

In [101]:
for shot in single_bias_ds.shot_number.values:
    shot_ds = single_bias_ds.sel(shot_number=shot).max('ts_radial_pos')
    
    for i, probe in enumerate(shot_ds.probe.values):
        fig, axes = plt.subplots(2)
        probe_ds = shot_ds.sel(probe=probe)
        
        spectrum = np.fft.fft(probe_ds['current'].values)
        freq = np.fft.fftfreq(len(probe_ds['current']), 0.0001)
        pstve_f = np.where(freq > 0)
        
        axes[0].semilogy(freq[pstve_f], np.abs(spectrum)[pstve_f], 'x', label='FFT Spectrum')
        axes[0].set_ylabel('Amplitude')
        axes[0].grid(which='both', axis='both')
        axes[0].set_xlim(0.01)
#         axes[0].set_ylim(1e-6, 1)
        axes[0].legend()

It doesn't look like there's anything in teh spectrum but this is the downsampled dataset, creating a new notebook to look in more detail

---
### All S Shots For Multiple B-Fields/Species (1.2T, H)

In [37]:
indices_08T_D = [462,463,464,465,466,467,468,469,470,471,472,475,476,477,478,479,480,481,482,483,484,485]
indices_12T_He = [245, 246, 247, 248, 249, 250, 251, 252, 254, 255, 256, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271]
indices_08T_He = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]
indices_15T_H = [490,491,492,493,494,495,496,497,498,499,500,503,504,505,506,507,509,510,511,512,513,514]
indices_12T_H = indices = [320, 321, 322, 323, 324, 325, 327, 328, 329, 330, 331, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346]
indices_08T_H = [41,42,43,44,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63]
indices_sshots = indices_08T_H + indices_12T_H + indices_15T_H + indices_08T_He + indices_12T_He + indices_08T_D

In [40]:
all_shots_ds, tilt_da = get_dataset_from_indices(indices_sshots, anglescan_fl=False, path_to_analysed_datasets=path_to_analysed_datasets)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [41]:
all_shots_ds

<xarray.Dataset>
Dimensions:                 (probe: 4, shot_number: 132, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * shot_number             (shot_number) int32 36 37 38 39 ... 481 482 483 484
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 -0.0 1.0 2.0 ... 2.0 1.0 0.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 nan nan ... nan
    current                 (shot_number, probe, time) float64 nan nan ... nan
    shot_time               (shot_number, probe, time) float64 nan nan ... nan
    start_t

In [42]:
sshots_ds = all_shots_ds.sel(probe='S')
sshots_ds = sshots_ds.where(np.isfinite(sshots_ds['voltage']), drop=True)
sshot_numbers = sshots_ds.shot_number

In [43]:
sshots_ds

<xarray.Dataset>
Dimensions:                 (shot_number: 66, time: 50, ts_radial_pos: 46)
Coordinates:
    probe                   <U1 'S'
  * shot_number             (shot_number) int32 36 37 38 39 ... 469 470 471 472
    ts_number               (shot_number) float64 29.0 30.0 31.0 ... 575.0 576.0
    ts_timestamp            (shot_number) float64 6.696e+18 ... 6.7e+18
    ts_time                 (shot_number) datetime64[ns] 2019-05-29T11:14:59.804525 ... 2019-06-07T12:20:36.209203
    adc_index               (shot_number) float64 41.0 42.0 43.0 ... 471.0 472.0
    adc_time                (shot_number) datetime64[ns] 2019-05-29T11:14:57 ... 2019-06-07T12:20:34
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 -0.0 1.0 2.0 ... 2.0 1.0 0.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, time) float64 -94.65 -92.53 ... -98.92
    current                 (shot_n

In [44]:
fit_df = fit_densscan_ds(all_shots_ds, probes=('S'), threshold=0.15, ax=True)

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/projects/flopter/flopter/core/fitters.py:216: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + (a * np.float_power(np.absolute(V), [0.75])))


Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no le

/home/jleland/coding/projects/flopter/flopter/core/fitters.py:216: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + (a * np.float_power(np.absolute(V), [0.75])))
/home/jleland/coding/projects/flopter/flopter/core/fitters.py:216: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + (a * np.float_power(np.absolute(V), [0.75])))


Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...
Time has no length, continuing...


In [45]:
fit_df

,shot,probe,B,ts_temp,ts_dens,shot_time,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f,dens,d_dens,chi2,reduced_chi2
0,36,S,0.8,1.2098421995176578,2.1372760899109834e+20,2019-05-29 11:14:59.804525,True,15.588014,10.084789,0.028982,0.024207,0.030583,0.337218,-5.314444,1.575925,6.513818e+18,6.220233e+18,1541.931667,33.520254
1,37,S,0.8,1.2298729758178688,2.2678454706544994e+20,2019-05-29 11:21:52.777115,True,2.070072,1.204879,0.000307,0.001074,31.478625,96.354657,-4.459868,0.143814,1.338624e+17,4.707206e+17,17933.928566,389.868012
2,38,S,0.8,1.174945517425577,2.4027974167854003e+20,2019-05-29 11:25:48.878599,True,2.862495,0.154824,0.138384,0.005201,0.025505,0.006724,-8.925142,0.056081,3.941155e+19,7.653674e+18,231.902955,5.946230
3,39,S,0.8,1.2009836929244544,2.3352615423264686e+20,2019-05-29 11:27:55.639395,True,2.975194,0.190412,0.179707,0.007998,0.019549,0.007443,-8.655104,0.053316,4.061182e+19,6.672512e+18,351.401971,9.010307
4,40,S,0.8,1.2008061398984395,2.297045332894742e+20,2019-05-29 11:30:33.490389,True,2.487622,0.115904,0.149773,0.004431,0.023155,0.004295,-8.490302,0.038840,3.100545e+19,4.232524e+18,184.494118,4.730618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,468,S,0.8,1.0194342697959007,5.709391632384068e+20,2019-06-07 12:15:09.987074,True,0.759359,0.050245,0.369122,0.000808,0.007991,0.000384,0.068856,0.076812,1.383066e+20,1.871797e+19,0.320904,0.006976
62,469,S,0.8,0.9744261932882826,5.60416643446544e+20,2019-06-07 12:16:25.367565,True,1.458575,0.222311,0.335770,0.001184,0.014303,0.001621,1.484106,0.462788,1.083731e+20,1.871114e+19,0.646406,0.014052
63,470,S,0.8,0.9772062148489283,5.6015976234592344e+20,2019-06-07 12:18:21.858326,True,1.033298,0.131221,0.300414,0.000720,0.011869,0.001126,0.462299,0.275879,1.424027e+20,2.834025e+19,0.301218,0.006548
64,471,S,0.8,0.9981601376374458,5.543591963946144e+20,2019-06-07 12:19:25.408740,True,0.400968,0.244370,0.241724,0.000797,0.006999,0.003240,-1.075962,0.554070,2.393624e+20,9.292590e+19,0.109689,0.002385


In [46]:
sshots_fit_ds = fit_df.to_xarray().rename(shot='shot_number').set_coords(['probe', 'shot_number']).swap_dims({'index':'shot_number'}).drop('index').drop('shot_time')


In [47]:
sshots_ds = all_shots_ds.sel(probe='S', shot_number=sshot_numbers)
sshots_ds = xr.merge([sshots_fit_ds, sshots_ds])
sshots_ds

<xarray.Dataset>
Dimensions:                 (shot_number: 66, time: 50, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) object 36 37 38 39 ... 469 470 471 472
    probe                   (shot_number) object 'S' 'S' 'S' 'S' ... 'S' 'S' 'S'
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 -0.0 1.0 2.0 ... 2.0 1.0 0.0
Dimensions without coordinates: time
Data variables:
    B                       (shot_number) float64 0.8 0.8 0.8 ... 0.8 0.8 0.8
    ts_temp                 (shot_number) object 1.2098421995176578 ... 0.9792942804401278
    ts_dens                 (shot_number) object 2.1372760899109834

In [48]:
B = sshots_ds.shot_b_field.round(2).values
H = np.where(sshots_ds.shot_hydrogen_gf.round().values > 0, 1, 0)
He = np.where(sshots_ds.shot_helium_gf.round().values > 0, 2, 0)
D = np.where(sshots_ds.shot_deuterium_gf.round().values > 0, 3, 0)
species_labels = ['None', 'H', 'He', 'D']
bfield_species_labels = [f'{B[i]}T_{species_labels[species]}' for i, species in enumerate(H+He+D)]
bfield_species_index = set(bfield_species_labels)
np.shape(bfield_species_labels)

(66,)

In [49]:
b_species_da = xr.DataArray(bfield_species_labels, dims='shot_number')
b_species_da

<xarray.DataArray (shot_number: 66)>
array(['0.8T_H', '0.8T_H', '0.8T_H', '0.8T_H', '0.8T_H', '0.8T_H',
       '0.8T_H', '0.8T_H', '0.8T_H', '0.8T_H', '0.8T_H', '1.2T_H',
       '1.2T_H', '1.2T_H', '1.2T_H', '1.2T_H', '1.2T_H', '1.2T_H',
       '1.2T_H', '1.2T_H', '1.2T_H', '1.2T_H', '1.5T_H', '1.5T_H',
       '1.5T_H', '1.5T_H', '1.5T_H', '1.5T_H', '1.5T_H', '1.5T_H',
       '1.5T_H', '1.5T_H', '1.5T_H', '0.8T_He', '0.8T_He', '0.8T_He',
       '0.8T_He', '0.8T_He', '0.8T_He', '0.8T_He', '0.8T_He', '0.8T_He',
       '0.8T_He', '0.8T_He', '1.2T_He', '1.2T_He', '1.2T_He', '1.2T_He',
       '1.2T_He', '1.2T_He', '1.2T_He', '1.2T_He', '1.2T_He', '1.2T_He',
       '1.2T_He', '0.8T_D', '0.8T_D', '0.8T_D', '0.8T_D', '0.8T_D',
       '0.8T_D', '0.8T_D', '0.8T_D', '0.8T_D', '0.8T_D', '0.8T_D'],
      dtype='<U7')
Dimensions without coordinates: shot_number

In [50]:
sshots_ds = sshots_ds.assign(bfield_species=b_species_da).swap_dims({'shot_number':'bfield_species'})
sshots_ds

<xarray.Dataset>
Dimensions:                 (bfield_species: 66, time: 50, ts_radial_pos: 46)
Coordinates:
    shot_number             (bfield_species) object 36 37 38 39 ... 470 471 472
    probe                   (bfield_species) object 'S' 'S' 'S' ... 'S' 'S' 'S'
    ts_number               (bfield_species) float64 ...
    ts_timestamp            (bfield_species) float64 ...
    ts_time                 (bfield_species) datetime64[ns] ...
    adc_index               (bfield_species) float64 ...
    adc_time                (bfield_species) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (bfield_species) float64 -0.0 1.0 2.0 ... 1.0 0.0
  * bfield_species          (bfield_species) <U7 '0.8T_H' '0.8T_H' ... '0.8T_D'
Dimensions without coordinates: time
Data variables:
    B                       (bfield_species) float64 0.8 0.8 0.8 ... 0.8 0.8 0.8
    ts_temp                 (bfield_species) object 1.209842

In [51]:
fig, ax = plt.subplots(3)
for bfsp in list(bfield_species_index):
#     if bfsp == '0.8T_D':
#         continue
    ax[0].errorbar('tilt', 'temp', yerr='d_temp', data=sshots_ds.sel(bfield_species=bfsp), label=bfsp)
    ax[1].errorbar('tilt', 'dens', yerr='d_dens', data=sshots_ds.sel(bfield_species=bfsp), label=bfsp)
    ax[2].plot('tilt', 'reduced_chi2', data=sshots_ds.sel(bfield_species=bfsp), label=bfsp)
ax[0].set_ylabel('T / eV')
ax[0].set_ylim(0, 15)

ax[1].set_ylabel(r'$n_e$ / m$^{-3}$')
ax[1].set_yscale('log')

# Plot the reduced chi^2 on a log plot and indicate where 1 is
ax[2].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[2].set_ylabel(r'$\chi^2_{\nu}$')
ax[2].set_xlabel(r'probe tilt / $^{\circ}$')
ax[2].set_yscale('log')

ax[0].legend()
fig.suptitle('Anglescan Fits for All S Probe Shots')

Text(0.5, 0.98, 'Anglescan Fits for All S Probe Shots')